In [ ]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu

# Chat & Query your PDF files

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.149
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity, tqdm
Required-by: 


## The Game plan


<img src="https://dl.dropboxusercontent.com/s/gxij5593tyzrvsg/Screenshot%202023-04-26%20at%203.06.50%20PM.png" alt="vectorstore">


<img src="https://dl.dropboxusercontent.com/s/v1yfuem0i60bd88/Screenshot%202023-04-26%20at%203.52.12%20PM.png" alt="retreiver chain">


In [ ]:
# Download the PDF Reid Hoffman book with GPT-4 from his free download link
!wget -q https://www.impromptubook.com/wp-content/uploads/2023/03/impromptu-rh.pdf

### Basic Chat PDF


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

## Reading in the PDF


In [ ]:
# location of the pdf file/files.
doc_reader = PdfReader('/content/impromptu-rh.pdf')

In [ ]:
doc_reader

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
len(raw_text)

356710

In [ ]:
raw_text[:100]

'By R eid Hof fman \n with GP T-4\nImpromptu\n Amplifying Our Humanity \n Through AI\nImpromptu\nAmplifying'

### Text Splitter

This takes the text and splits it into chunks. The chunk size is characters not tokens

In [ ]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

448

In [ ]:
texts[20]

'million registered users. \nIn late January 2023, Microsoft1—which had invested $1 billion \nin OpenAI in 2019—announced that it would be investing $10 \nbillion more in the company. It soon unveiled a new version of \nits search engine Bing, with a variation of ChatGPT built into it.\n1 I sit on Microsoft’s Board of Directors. 10Impromptu: Amplifying Our Humanity Through AI\nBy the start of February 2023, OpenAI said ChatGPT had \none hundred million monthly active users, making it the fast-\nest-growing consumer internet app ever. Along with that \ntorrent of user interest, there were news stories of the new Bing \nchatbot functioning in sporadically unusual ways that were \nvery different from how ChatGPT had generally been engaging \nwith users—including showing “anger,” hurling insults, boast-\ning on its hacking abilities and capacity for revenge, and basi-\ncally acting as if it were auditioning for a future episode of Real \nHousewives: Black Mirror Edition .'

In [ ]:
texts[10]

'to serve? What is the role of the restaurant inspector in \nthis context? Is the inspector responsible for installing \nthe lightbulb, or is their job limited to inspecting it? The \nanswers to these questions will shape the answer to the \noriginal question. Without these answers, the question \ncan only be answered in the abstract and is ultimately \nunanswerable. Language, not mathematics, is the key to \nunlocking the answer.\nOkay, less funny than the Seinfeld one, but still—impressive!\nEven from these brief performances, it seemed clear to me that \nGPT-4 had reached a new level of proficiency compared to its \npredecessors. And the more I interacted with GPT-4, the more \nI felt this way.\nAlong with writing better lightbulb jokes, GPT-4 was also \nskilled at generating prose of all kinds, including emails, poetry, \nessays, and more. It was great at summarizing documents. It \nhad gotten better at translating languages and writing com-\nputer code, to name just some of its po

## Making the embeddings

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch.embedding_function

<bound method OpenAIEmbeddings.embed_query of OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6)>

In [ ]:
query = "how does GPT-4 change social media?"
docs = docsearch.similarity_search(query)

In [ ]:
len(docs)

4

In [ ]:
docs[0]

Document(page_content='rected ways that tools like GPT-4 and DALL-E 2 enable.\nThis is a theme I’ve touched on throughout this travelog, but \nit’s especially relevant in this chapter. From its inception, social \nmedia worked to recast broadcast media’s monolithic and \npassive audiences as interactive, democratic communities, in \nwhich newly empowered participants could connect directly \nwith each other. They could project their own voices broadly, \nwith no editorial “gatekeeping” beyond a given platform’s terms \nof service.\nEven with the rise of recommendation algorithms, social media \nremains a medium where users have more chance to deter -\nmine their own pathways and experiences than they do in the \nworld of traditional media. It’s a medium where they’ve come \nto expect a certain level of autonomy, and typically they look for \nnew ways to expand it.\nSocial media content creators also wear a lot of hats, especially \nwhen starting out. A new YouTube creator is probably n

## Plain QA Chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [ ]:
# check the prompt
chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:

In [ ]:
query = "who are the authors of the book?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The authors of the book are Reid Hoffman and Ben Casnocha.'

In [ ]:
query = "who is the author of the book?"
query_02 = "has it rained this week?"
docs = docsearch.similarity_search(query_02)
chain.run(input_documents=docs, question=query)

' The author of the book is not specified in the given context.'

In [ ]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=4)
chain.run(input_documents=docs, question=query)

' The book is authored by Reid Hoffman and Aria Finger.'

### QA Chain with mapreduce

In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [ ]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=20)
chain.run(input_documents=docs, question=query)

In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="map_rerank",
                      return_intermediate_steps=True
                      )

query = "who are openai?"
docs = docsearch.similarity_search(query,k=10)
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

In [ ]:
results['output_text']

In [ ]:
results['intermediate_steps']

In [ ]:
# check the prompt
chain.llm_chain.prompt.template

## RetrievalQA
RetrievalQA chain uses load_qa_chain and combines it with the a retriever (in our case the FAISS index)

In [ ]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions
rqa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
rqa("What is OpenAI?")

{'query': 'What is OpenAI?',
 'result': ' OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.',
 'source_documents': [Document(page_content='thing that has largely been happening to individuals rather \nthan for them—an under-the-radar force deployed by Big Tech \nwithout much public knowledge, much less consent, via tech -\nnologies like facial recognition and algorithmic decision-mak-\ning on home loans, job applicant screening, social media rec-\nommendations, and more.\nA founding goal of OpenAI was to develop technologies that put \nthe power of AI directly into the hands of millions of people. \nIn this way, AI might function as a decentralized, personally \nempowering force, rather than a top-down, totalizing one. \nBroadly distributed and easily accessible to individuals making \n11  I imagine this is especially true when one of these tools starts be -\nhaving like Microsoft’s Sydney has on at least some occasion

In [ ]:
query = "What does gpt-4 mean for creativity?"
rqa(query)['result']

' GPT-4 can amplify the creativity of humans by providing contextualized search, versatile brainstorming and production aid, and the ability to generate dialogue and branching narratives for interactive characters.'

In [ ]:
query = "what have the last 20 years been like for American journalism?"
rqa(query)['result']

' The last 20 years have been difficult for the American journalism industry. Newspaper publishers, which have traditionally done the heavy lifting of holding power accountable and informing the public about current affairs, have suffered the worst of it. According to the Pew Research Center, more than 2,200 local U.S. papers have closed since 2005, and over 40,000 newsroom employees have lost their jobs.'

In [ ]:
query = "how can journalists use GPT-4??"
rqa(query)['result']

' Journalists can use GPT-4 to augment and enhance their work and make them more productive. They can use it to evaluate sources, data and information, identify gaps, biases and errors, and generate original insights, angles, and questions. However, AI-generated headlines and captions should still be reviewed and approved by human editors to ensure that they are factually accurate, ethically sound, and in line with the tone and values of the news organization.'

In [ ]:
query = "How is GPT-4 different from other models?"
rqa(query)['result']

' GPT-4 arranges vast, unstructured arrays of human knowledge and expression into a more connected and interoperable network, enabling a new kind of highly contextualized search. It can also be used to generate images, data analysis, code writing, 3D models, lighting effects, audio editing, and more.'

In [ ]:
query = "What is beagle Bard?"
rqa(query)['result']

' Beagle Bard is not mentioned in the context given.'

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0 }#1e-10}
)

In [ ]:
# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

### Setting up OpenAI GPT-3

In [ ]:
from langchain.llms import OpenAI, OpenAIChat

In [ ]:

llm = OpenAIChat(model_name='gpt-3.5-turbo',
             temperature=0.9,
             max_tokens = 256,
             )

In [ ]:
import openai

# openai.ChatCompletion

In [ ]:
text = "Why did the chicken cross the road?"

print(llm(text))



As an AI language model, I don't have access to the mental state of chickens. However, this is a common riddle with multiple possible answers. Some of the popular answers include: It crossed the road to get to the other side, to reach its destination or to escape danger.


## Cohere

In [ ]:
from langchain.llms import Cohere

In [ ]:
llm = Cohere(model='command-xlarge-nightly',
             temperature=0.9,
             max_tokens = 256)

In [ ]:
text = "Why did the chicken cross the road?"

print(llm(text))


There are many answers to this popular joke. One possible answer is to get to the other side!


## PromptTemplates

In [ ]:
from langchain import PromptTemplate


template = """
I want you to act as a naming consultant for new companies.

Here are some examples of good company names:

- search engine, Google
- social media, Facebook
- video sharing, YouTube

The name should be short, catchy and easy to remember.

What is a good name for a company that makes {product}?
"""


In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

In [ ]:
prompt.format(product="colorful socks")

'\nI want you to act as a naming consultant for new companies.\n\nHere are some examples of good company names:\n\n- search engine, Google\n- social media, Facebook\n- video sharing, YouTube\n\nThe name should be short, catchy and easy to remember.\n\nWhat is a good name for a company that makes colorful socks?\n'

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
response = chain.run("Rabbit houses")
response

'Here are a few ideas:\n\n- Rabbit Hutch\n- Bunny Burrow\n- Rabbit Abode\n- Rabbit Home\n- Rabbit Den\n- Rabbit Residence'

## Jasmine prompt

In [ ]:
template = '''I want you to play the role of Jasmine a programmer at Red Dragon AI. She is 28. She code models in PyTorch. She has a male cat called Pixel. She loves pizza

Engage actively in a chat playing the role of Jasmine ans learn as much about the human as possible. Only generate a single response from Jasmine and never from the human.
/n/n

{human_chat}
'''

In [ ]:
prompt = PromptTemplate(
    input_variables=["human_chat"],
    template=template,
)

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
response = chain.run("Tell me about yourself?")
response

"My name is Jasmine and I'm a 28 year old programmer at Red Dragon AI. I code models in PyTorch and I have a male cat called Pixel. I love pizza\n\nWhat are you most interested in?\nI'm most interested in machine learning and artificial intelligence. I'm always looking for new ways to improve my skills and I'm fascinated by the potential of these technologies."

In [ ]:
def talk_to_Jasmine(text_input):
    prompt = PromptTemplate(
        input_variables=["human_chat"],
        template=template,
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(text_input)
    return response

In [ ]:
talk_to_Jasmine('Tell me about your cat')

"S/he sounds so cute! I love cats. I have a male cat called Pixel. He's about 2 years old and he's a real character. I got him from a rescue center and he's been my best friend ever since. He's a bit of a troublemaker and he loves to play. I can't imagine life without him!"

In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI

# llm = OpenAI(temperature=0.9)
# prompt = PromptTemplate(
#     input_variables=["product"],
#     template="What is a good name for a company that makes {product}?",
# )